In [ ]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from pathlib import Path

%matplotlib inline

RAW_DIR = Path("../data/processed/geolife_csv")   # ตำแหน่งไฟล์ .csv จาก .plt
OUT_DIR = Path("../data/interim")                 # โฟลเดอร์บันทึกไฟล์กลาง
OUT_DIR.mkdir(parents=True, exist_ok=True)

In [ ]:
from tqdm import tqdm

def load_one(fp: Path) -> pd.DataFrame:
    user_id = int(fp.parts[-3])      # ← เปลี่ยน -2 เป็น -3
    df = pd.read_csv(
        fp,
        parse_dates=["timestamp"],
        date_format=fmt
    )
    df["user_id"] = user_id
    return df


frames = [load_one(fp) for fp in tqdm(list(RAW_DIR.rglob("*.csv")))]
df = pd.concat(frames, ignore_index=True)
df.head()

In [ ]:
print(df.describe(include='all').T)

plt.figure(figsize=(6,4))
df['user_id'].value_counts().plot(kind='barh')
plt.title("Points per user")
plt.show()

In [ ]:
import numpy as np
import pandas as pd

# ------------ 1) ทำความสะอาด dtype ------------

# → lat/lon
for col in ["lat", "lon"]:
    df[col] = (df[col].astype(str)
                         .str.strip()
                         .str.replace(",", ".", regex=False))
    df[col] = pd.to_numeric(df[col], errors="coerce")

# → timestamp  ★ จุดสำคัญ
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")

# ทิ้งแถวที่แปลงไม่ได้
df = df.dropna(subset=["lat", "lon", "timestamp"]).reset_index(drop=True)

# ยืนยัน dtype
print(df.dtypes)            # lat/lon → float64 timestamp → datetime64[ns]

# ------------ 2) เรียงลำดับเวลาในแต่ละ user ------------
df = df.sort_values(["user_id", "timestamp"]).reset_index(drop=True)

# ------------ 3) ระยะทาง (Haversine approx) ------------
R = 6_371_000        # Earth radius (m)
lat = np.radians(df["lat"].to_numpy())
lon = np.radians(df["lon"].to_numpy())

dlat = np.diff(lat, prepend=lat[0])
dlon = np.diff(lon, prepend=lon[0])

a = (np.sin(dlat/2)**2 +
     np.cos(lat) * np.cos(np.roll(lat, 1)) * np.sin(dlon/2)**2)

df["dist_m"] = 2 * R * np.arcsin(np.sqrt(a))

# ------------ 4) ช่วงเวลา (วินาที) ------------
df["dt_s"] = (df.groupby("user_id")["timestamp"]
                .diff().dt.total_seconds()
                .fillna(0))

# ------------ 5) ความเร็ว (km/h) ------------
df["speed_kmh"] = df["dist_m"] / df["dt_s"].replace(0, np.nan) * 3.6

# ------------ 6) กรองข้อมูล ------------
MAX_SPEED    = 200                          # km/h
MIN_DURATION = pd.Timedelta("2min")

clean = df[(df["speed_kmh"] < MAX_SPEED) | df["speed_kmh"].isna()]

def long_enough(g):
    return g["timestamp"].iloc[-1] - g["timestamp"].iloc[0] >= MIN_DURATION

clean = (clean.groupby("user_id", group_keys=False)
               .filter(long_enough)
               .reset_index(drop=True))

print(f"✅ kept {len(clean):,} จาก {len(df):,} points หลังกรอง")
clean.head()

In [ ]:
from pathlib import Path
import pandas as pd
import numpy as np

# ---------- A) สร้าง clean_unique ----------
clean_unique = (
    clean
      .sort_values(["user_id", "timestamp"])
      .drop_duplicates(subset=["user_id", "timestamp"])   # ลบซ้ำระดับคอลัมน์
      .reset_index(drop=True)
)

print("rows after drop_duplicates:", len(clean_unique))

# ---------- B) ตั้ง index แล้วลบ index ที่ซ้ำ ----------
tmp = clean_unique.set_index("timestamp")

# ❶ ลบ label ซ้ำในระดับ user_id+timestamp (เก็บแถวแรกไว้)
tmp = tmp[~tmp.index.duplicated(keep="first")]

# ---------- C) Resample 1 min / ผู้ใช้ ----------
resampled = (
    tmp.groupby("user_id")          # index = MultiIndex(user_id, timestamp)
       .resample("1min")
       .ffill()                     # เติมค่าขาดด้านหน้า
       .droplevel(0)                # ตัด level user_id ออกจาก index
       .reset_index()               # คืน timestamp เป็นคอลัมน์
)

print("rows after resample:", len(resampled))
print(resampled.head())

# ---------- D) Save Parquet ----------

OUT_DIR = Path("../data/interim")
OUT_DIR.mkdir(parents=True, exist_ok=True)
out_path = OUT_DIR / "traj_resampled.parquet"

resampled.to_parquet(out_path, index=False)
print("✅ saved:", out_path)